In [5]:
from .TSP_Formulation_Methods import *
import numpy as np
import glob

distances_original_matrix = np.loadtxt("./deliverable/data/matriz-rutas-granada")

ImportError: attempted relative import with no known parent package

# First try: set all lambdas as the upper bound of the cost matrix

In [ ]:
N = 6 # Number of stops
p = 2 # Number of travels, aka number of edges. The number of involucred stops is then p+1
startNode = 1
endNode = 3
distances_N_stops_normalized = distances_original_matrix[:N,:N]/np.max(distances_original_matrix[:N,:N])

lambdas = [calculate_upper_bound_distances(distances_N_stops_normalized, p) for i in range(5)]
Q_matrix,_ = create_QUBO_matrix(distances_N_stops_normalized, p, startNode, endNode, lambdas)
combinations_zipped = brute_force_finding(Q_matrix, distances_N_stops_normalized, p)
minimal_solution = np.array(list(combinations_zipped[0][0]), dtype=int)

show_parameters_of_solution(minimal_solution, distances_N_stops_normalized, N, p, startNode, endNode)

draw_solution_graph(minimal_solution, distances_N_stops_normalized, p, startNode, endNode)

plot_brute_force_minimums(combinations_zipped, N, p, startNode, endNode, rangePlot=20)
    

In [ ]:
# Analysis

# Compute the relative diference between the minimum and the second best solution

minimal_solution_cost = combinations_zipped[0][1]
second_best_solution_cost = combinations_zipped[1][1]
relative_difference_two_bests = np.abs((second_best_solution_cost - minimal_solution_cost)/minimal_solution_cost)

print("The relative difference between the minimum and the second best valid solution is: ", relative_difference_two_bests)

# Compute the relative difference between the minimal solution and the first invalid solution (index 5 in this case)

first_invalid_solution_cost = combinations_zipped[5][1]
relative_difference_minimal_first_invalid = np.abs((first_invalid_solution_cost - minimal_solution_cost)/minimal_solution_cost)

print("The relative difference between the minimal solution and the first invalid solution is: ", relative_difference_minimal_first_invalid)

# CONCLUSIONS:
# The upper bound is a valid approximation to find valid solutions, however, the minimums have almost the same energy.
# We aim to optimize the lambdas to reduce the (absolute value of the) total energy of the optimal solution

## CONCLUSIONS:
The upper bound is a valid approximation to find valid solutions, however, the minimums have almost the same energy.
We aim to optimize the lambdas to reduce the (absolute value of the) total energy of the optimal solution

# Generation of optimized lambdas
Methods to generate optimized lambdas (weights) for the QUBO matrix. Below of this systematic search there is a simple example of how to use the optimized lambdas to solve the TSP problem.


In [6]:
# This is a general approach to produce the optimized lambdas for all the possible combinations of N and p
import multiprocessing as mp

min_N = 1
max_N = 6
min_p = 1
max_p = 3

iterations = 5
best_combinations_number = 30 # number of best solutions to count for the optimization
factor_over_upper_bound = 0.01

for N in range(min_N, max_N+1):
    for p in range(min_p, min(max_p+1, N)):
        L = 1
        all_start_end_combinations = generate_all_start_end_combinations(N, L)
        distances_N_stops_normalized = distances_original_matrix[:N,:N]/np.max(distances_original_matrix[:N,:N])

        args = []
        for startNodes, endNodes in all_start_end_combinations:
            startNode = startNodes[0]
            endNode = endNodes[0]
            args.append((distances_N_stops_normalized, p, startNode, endNode, iterations, best_combinations_number, factor_over_upper_bound))

        def optimize_and_save(*args):
            distances_N_stops_normalized, p, startNode, endNode, iterations, best_combinations_number, factor_over_upper_bound = args
            _ , optimized_lambdas, combinations_zipped = optimize_lambdas(
                distances_N_stops_normalized,
                p, startNode, endNode, 
                iterations, 
                best_solution_number=best_combinations_number, factor_over_upper_bound=factor_over_upper_bound)

            minimal_solution = np.array(list(combinations_zipped[0][0]), dtype=int)
            if check_solution_return(minimal_solution, N, p, startNode, endNode):
                np.savetxt("./data/lamdasOptimized/lambdas_N_{}_p_{}_startNode_{}_endNode_{}_iterations_{}".format(N, p,startNode,endNode, iterations ), optimized_lambdas, delimiter=",")
            else:
                print("Solution not valid for N={}, p={}, startNode={}, endNode={}".format(N, p, startNode, endNode))

        with mp.Pool(mp.cpu_count()) as pool:
            pool.starmap(optimize_and_save, args)
            pool.close()
            pool.join()
        

KeyboardInterrupt: 

# Example of simple use

In [ ]:
# Global Parameters

N = 6 # Number of stops
p = 2 # Number of travels, aka number of edges. The number of involucred stops is then p+1
startNode = 1
endNode = 3

iterations = 5
best_combinations_number = 30 # number of best solutions to count for the optimization
factor_over_upper_bound = 0.001 # Factor over the upper bound to set the initial lambdas

# Process Parameters

p = min(p, N-1)
startNode = min(startNode, N-1)
endNode = min(endNode, N-1)
distances_N_stops_normalized = distances_original_matrix[:N,:N]/np.max(distances_original_matrix[:N,:N])


# Optimization

Q_matrix_optimized, optimized_lambdas, combinations_zipped = optimize_lambdas(distances_N_stops_normalized, p, startNode, endNode, iterations=iterations, best_solution_number=best_combinations_number, factor_over_upper_bound=factor_over_upper_bound)
optimized_solution,_ = solve_qubo_with_Dwave(Q_matrix_optimized, num_reads=1000)

# Show solution

Q_matrix_optimized, optimized_solution_cost, optimized_solution_total_cost = show_parameters_of_solution(optimized_solution, distances_N_stops_normalized, N, p, startNode, endNode, optimized_lambdas)

print("\nQ matrix of optimized solution:")
print(Q_matrix_optimized)

draw_solution_graph(optimized_solution, distances_N_stops_normalized, p, startNode, endNode)

In [ ]:
# Check how distributed are the real minimas 

global_minimum = np.array(list(combinations_zipped[0][0]), dtype=int)

show_parameters_of_solution(global_minimum, distances_N_stops_normalized, N, p, startNode, endNode)

draw_solution_graph(global_minimum, distances_N_stops_normalized, p, startNode, endNode)

plot_brute_force_minimums(combinations_zipped, N, p, startNode, endNode, rangePlot=30)
print(startNode, endNode)

# In the final graph, red dots are the valid solutions and the crosses the distance cost of bidirectional routes.

In [ ]:
# Analysis

# Compute the relative diference between the minimum and the second best valid solution (adjust the index if needed)

minimal_solution_cost = combinations_zipped[0][1]
second_best_solution_cost = combinations_zipped[1][1]
relative_difference_two_bests = np.abs((second_best_solution_cost - minimal_solution_cost)/minimal_solution_cost)

print("The relative difference between the minimum and the second best valid solution is: ", relative_difference_two_bests)

# Compute the relative difference between the minimal solution and the first invalid solution (adjust the index if needed)

first_invalid_solution_cost = combinations_zipped[3][1]
relative_difference_minimal_first_invalid = np.abs((first_invalid_solution_cost - minimal_solution_cost)/minimal_solution_cost)

print("The relative difference between the minimal solution and the first invalid solution is: ", relative_difference_minimal_first_invalid)

## CONCLUSIONS:

 We started with a relatively low value for the lamdbas (0.001 * upper_bound) and did relatively few iterations (5) so we could not differentiate the valid solutions from each other or the valid solutions from the invalid ones. We could increase the number of iterations and the factor over the upper bound to get a better approximation of the optimal solution.

In the aim of differentiating the valid best solution from any other, setting the factor in 0.01 and do 5 iterations is enough.

In [ ]:
# Based on observations made in Analysis_Optimized_Lambdas, a good choice of general lambdas for all N and p can be computed as the maximum value for the first few N and p optimized lambdas


max_lambdas = compute_general_lambdas(distances_original_matrix, max_N=5, iterations=5, initial_factor=0.01)
print(max_lambdas)